 Install Dependencies

In [ ]:
!pip install facenet-pytorch faiss-cpu flask-ngrok tqdm


 Import Libraries

In [ ]:
from facenet_pytorch import InceptionResnetV1, MTCNN
from PIL import Image
import torch
import numpy as np
import faiss
import os
from tqdm import tqdm
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import io

Setup MTCNN (face detector) and FaceNet (embedding model)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(image_size=160, margin=0, device=device)
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

  0%|          | 0.00/107M [00:00<?, ?B/s]

Upload and unzip dataset (upload ZIP file manually)

In [ ]:
from google.colab import files
import shutil

print("Please upload your dataset ZIP file (structured by person folders)...")
uploaded = files.upload()
zip_name = list(uploaded.keys())[0]
shutil.unpack_archive(zip_name, 'dataset')

Please upload your dataset ZIP file (structured by person folders)...


Saving project_k (2).zip to project_k (2).zip


Build embedding index and label list

In [ ]:
embeddings = []
labels = []
paths = []

for root, dirs, files_in_dir in os.walk('dataset'):
    for file in files_in_dir:
        if file.lower().endswith(('jpg', 'jpeg', 'png')):
            img_path = os.path.join(root, file)
            try:
                img = Image.open(img_path).convert('RGB')
                face = mtcnn(img)
                if face is not None:
                    emb = model(face.unsqueeze(0).to(device)).squeeze(0).detach().cpu().numpy()
                    embeddings.append(emb)
                    labels.append(os.path.basename(root))
                    paths.append(img_path)
            except Exception as e:
                print(f"Error processing {img_path}: {e}")

embeddings = np.array(embeddings).astype('float32')
print(f"✅ Indexed {len(embeddings)} face images.")

✅ Indexed 93 face images.


Build Faiss Index

In [ ]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

Upload and Match Query Image

In [ ]:
# Step 0: Install dependencies and set up ngrok
!pip install -q pyngrok

from flask import Flask, request, jsonify
from pyngrok import ngrok, conf
from PIL import Image
import io
import torch
import numpy as np

# 🔑 Set your ngrok authtoken (you only need to do this once per session)
conf.get_default().auth_token = "2wczMgF7rlSM0EkZuX7LvgITQNf_7itKcLPyvqTW6qPSHZGtz"

# ✅ Make sure the following objects are already defined earlier:
# mtcnn, model, device, index, labels

# Step 1: Initialize Flask app
app = Flask(__name__)

# Step 2: Define the face identification endpoint
@app.route("/identify", methods=["POST"])
def identify():
    if 'image' not in request.files:
        return jsonify({"error": "No image uploaded"}), 400

    file = request.files['image']
    try:
        img = Image.open(io.BytesIO(file.read())).convert('RGB')
        face = mtcnn(img)

        if face is None:
            return jsonify({"found": False, "reason": "No face detected"}), 200

        # 🧠 Get face embedding
        with torch.no_grad():
            emb = model(face.unsqueeze(0).to(device)).squeeze(0).cpu().numpy()
        emb = np.expand_dims(emb.astype('float32'), axis=0)

        # 🔍 Search in FAISS index
        D, I = index.search(emb, k=1)
        threshold = 1.0

        if D[0][0] < threshold:
            return jsonify({
                "found": True,
                "label": labels[I[0][0]],
                "distance": float(D[0][0])
            })
        else:
            return jsonify({
                "found": False,
                "closest_distance": float(D[0][0])
            })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Step 3: Start ngrok tunnel and run the app
public_url = ngrok.connect(5000)
print("🚀 Public URL:", public_url)

app.run(port=5000)


🚀 Public URL: NgrokTunnel: "https://66ba-34-125-138-183.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [06/May/2025 04:34:55] "POST /identify HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 04:35:15] "POST /identify HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 04:35:34] "POST /identify HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 04:35:55] "POST /identify HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 04:36:15] "POST /identify HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 04:36:36] "POST /identify HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 04:36:58] "POST /identify HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 04:37:16] "POST /identify HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 04:37:36] "POST /identify HTTP/1.1" 200 -
